# FP management

In thi notebook we will define the various methods to perform changes to the database to keep track of the various events. 

The data source is the Statistiche_giocatori.pkl file, created by the IGNOBEL notebook, which needs to be updated prior any management done in this notebook.

In particular we will take care of:

- Adding new free agent players to the database, when they are included in the FC list (done)
- Assigning trophies (done)
- Assigning fines (done)
- Calculating total roster value to determine the salary, extra budget and luxury tax (done)
- Update team_real (done)
- UPDATE PLAYER NAME IF IT HAS CHANGED ON FC: (done)
    - Sometime FC changes the name of a player. In 2019/2020 LUKAKU was 'LUKAKU R.', now just 'LUKAKU', because Jordan left serie A. We need a way of scanning the players in our db by ID and update the name as needed. Maybe it can be included in the stats update algorithm
- Anything else?

It should be fairly easy to turn this into a dashboard using widgets, but that is for later

In [142]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import json
import copy
from pymongo import MongoClient
from pprint import pprint
import pymongo
import progressbar
with open('credential.json','r') as f:
    cred = json.load(f)
stats = pd.read_pickle('../IGNOBEL/Statistiche_giocatori.pkl')

## Update personal info
this part is to run through the players in our database and cross check the info with the stats db:
- if the player is not in our db, we don't care: we'll add it when someone wants to buy them
- if the player is in our db but not in stats it means that it left serie A, so the algorithm should change the team to none:
```collection.update_one({'_id': pl['_id']},{'$set':{'info.personal_info.team_real': None}})
```
- if the player has changed teams we need to update the information
- update name if it has changed, based on id

In [143]:
def update_personal_info(stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]

    player_out = []
    posts = collection.find({})
    stats.index = list(stats['Id'])
    for dic in posts:
        Id = dic['_id']
        if Id in stats.index:
            collection.update_one({'_id': Id},{'$set':{'name': stats.loc[Id].Nome }})
            collection.update_one({'_id': Id},{'$set':{'info.personal_info.team_real': stats.loc[Id].Squadra}})
        else:
            collection.update_one({'_id': Id},{'$set':{'info.personal_info.team_real': None}})
            player_out.append(collection.find_one({'_id': Id})['name'])
    print('Players no longer in serie A')
    return player_out

In [144]:
#This updates the personal info, name and team, based on the file stats produced in the IGNOBEL notebook.
#the function returns the list of players that are in our db but no longer in FC (e.g. those who retired or left serie a)
update_personal_info(stats)

/Users/lucaagozzino/opt/anaconda3/lib/python3.7/site-packages/pymongo/common.py:559: UserWarning: Unknown option retryWrites
  warnings.warn(str(exc))


Players no longer in serie A


['EMPEREUR']

## UPDATE stats

In [145]:
#updating the stats of a player based on id
def update_stats(stats, Id, CR = cred['cred'], DB = 'Game', CO = 'Players'):

    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    stats.index = list(stats.Id)
    stats_dict = dict(stats[stats['Id'] == int(Id)].T[Id][4:20])

    collection.update_one({'_id': Id}, {'$set': {'info.stats': stats_dict}})

In [146]:
#updating stats of all players in our db
def update_stats_all(stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    
    all_pl = list(collection.find({}))
    
    for pl in progressbar.progressbar(all_pl):
        #print(pl['_id'])
        if len(stats[stats['Id'] == pl['_id']]) == 0:
            print(pl['name']+' left serie A')
            #
            continue
        else:
            update_stats(stats, pl['_id'], CR, DB, CO)

In [147]:
# algorithm runs through the entire db and updates stats value
# if player no more in serie a it prints a message, but we need to include change of team_real to none
#it is probably best to 
update_stats_all(stats)

 65% (370 of 567) |##############        | Elapsed Time: 0:01:19 ETA:   0:00:36

EMPEREUR left serie A


100% (567 of 567) |######################| Elapsed Time: 0:02:03 Time:  0:02:03


## Add player to database
First, we create a list with all the players in the stats db who are not in our db

In [148]:
#this function returns the players in stats that are not in our db
def find_missing_ids(stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    
    stats.index = list(stats['Id'])
    our_db = collection.find({})
    our_ids = []
    for el in our_db:
        our_ids.append(el['_id'])
    missing_ids = []
    for ind in stats.index:
        if ind in our_ids:
            continue
        else:
            missing_ids.append(ind)
    
    return missing_ids

In [149]:
#this adds a player to our db from the stats db
def add_player(Id, stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    # Dictionary with player current team and loan info
    CurrentTeamDict = {
        'owner': None, #luca, pietro etc
        'squad': None, # 'Main' or 'Primavera'
        'start_date': '',
        'previous_team': None, #last team (e.g. owner, squad)
        'quotation_initial': 0,
        'on_loan': False, #True or False
        'loan_info': None
    }

    # Dictionary with player ownership info
    ContractDict = {
        'owner': None, #this seems redundant
        'start_date': '',
        'cost': 0,
        'acquisition_mode': None, #asta_svincolati, draft, acquisto
        'previous_owner': None, #owner or None
        'quotation_initial': 0
    }

    # Dictionary with player personal info
    PersonalInfoDict = {
        'full_name': '',
        'birthdate': '01/01/1970',
        'nation': '',
        'team_real': '',
        'FC_role': ''
    }

    # Nested dictionary for a single player info
    PlayerDict = {
        '_id': 0,
        'name': '',
        'personal_info': PersonalInfoDict,
        'contract': ContractDict,
        'current_team': CurrentTeamDict,
    }
    transStat = stats.T
    PlayerDict['_id'] = Id
    PlayerDict['name'] = transStat[Id]['Nome']
    PlayerDict['personal_info']['FC_role'] = transStat[Id]['R']
    PlayerDict['personal_info']['team_real'] = transStat[Id]['Squadra']
    PlayerDict['personal_info']['full_name'] = transStat[Id]['Nome Completo']
    PlayerDict['personal_info']['nation'] = transStat[Id]['Nazionalit\'a']
    PlayerDict['personal_info']['birthdate'] = transStat[Id]['Classe']
    PlayerDict['contract']['quotation_initial'] = transStat[Id]['Qt. I']
    PlayerDict['contract']['owner'] = None
    PlayerDict['current_team']['owner'] = None
    PlayerDict['current_team']['quotation_initial'] = transStat[Id]['Qt. I'] 
    temp = dict(transStat[Id][4:20])
    temp['Qt_I'] = temp.pop('Qt. I')
    temp['Qt_A'] = temp.pop('Qt. A')
    PlayerDict['stats'] = temp
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    
    collection.insert_one(PlayerDict)
    
    return PlayerDict

In [150]:
missing_ids = find_missing_ids(stats)
missing_ids 

[]

In [151]:
#this function adds all the players in the missing_ids list.

In [152]:
def add_multi_pl(Ids, stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    pls = []
    for Id in Ids:
        pl = add_player(Id, stats, CR, DB, CO)
        pls.append(pl['name'])
    return pls

In [23]:
#The first round returned these four players, which now are in our db, so running the missing_id function returns an empty list
add_multi_pl(missing_ids, stats)

['RUGGERI', 'BALOGH', 'UDOGIE', 'EYSSERIC']

## Manage roster value and extra budget/luxury tax

This class acts directly on the full database, but we could in principle create a session class, like we did for the market operations, and then close the session by copying everything only at the end, but since the db has only 8 entries I don't see the point, it is easy to check and fix stuff

In [153]:
class manager:
    
    def __init__(self, owner, stats = stats, CR = cred['cred'], DB = 'Game', CO = 'Managers', CP = 'Players'):
        
        self.par = CR, DB, CO
        cluster = MongoClient(CR)
        db = cluster[DB]
        self.collection = db[CO]
        self.coll_P = db[CP]
        
        self.dic = self.collection.find_one({'owner': owner})
        self.dic_p = self.coll_P.find(({'info.contract.owner': owner,'info.current_team.squad':'main'}))
        
        pl_value = 0
        for pl in self.dic_p:
            pl_value += pl['info']['stats']['Qt. A']
        
        self.main_roster_value = pl_value
        
        self.owner = self.dic['owner']
        self.team_name = self.dic['team_name']
        self.budget = self.dic['budget']
        self.palmares = self.dic['palmares']
        self.total_wins = self.dic['total_wins']
    
    def modify_budget(self, delta):
        
        self.collection.update_one({'owner': self.owner},{'$inc':{'budget':delta}})
        self.budget = self.budget + delta #otherwise you have to re-run the cell to see the modification
    
    def assign_trophy(self, trophy, season, prize_money):
        
        dic = { 
            'Type': trophy,
            'Prize (EUR)': prize_money,
            'Season': season
              }
        
        self.collection.update_one({'owner': self.owner},{'$addToSet': {'palmares': dic}})
        self.collection.update_one({'owner': self.owner},{'$inc': {'total_wins': prize_money}})
        
        self.palmares.append(dic)
        self.total_wins += prize_money
    
    def assign_fine(self, fine = 4, reason = 'Missing lineup'):
        d = date.today()
        
        dic = { 
            'Reason': reason,
            'Fine (EUR)': fine,
            'date': d.strftime('%Y/%m/%d')
              }
        self.collection.update_one({'owner': self.owner},{'$addToSet': {'fines': dic}})
    
    def assign_extra_budget(self, extra_budget = 100, salary_cap = 350):
        
        temp = extra_budget + salary_cap - self.main_roster_value
        
        extra = max(0, temp)
        self.collection.update_one({'owner': self.owner},{'$inc': {'budget': extra}})
        
        if temp < 0:
            print(self.owner+ ' exceeded the luxury tax limit. He will paying '+ str(-temp)+ ' EUR to the league.')

In [138]:
luca = manager('luca')

In [154]:
#THIS assigns extra budget with the formula: Extra budget = 100 + salary_cap - roster_value (see method definition)
#luca.assign_extra_budget()

In [78]:
enzo = manager('enzo')

In [79]:
#Assign prize (already done, so don't repeat otherwise there are duplicates in the db)
#enzo.assign_trophy('Coppa di Lega', '20/21', 25)

In [94]:
nanni = manager('nanni')
mario = manager('mario')

#Fines have been already assigned, so do not run again
#nanni.assign_fine()
#mario.assign_fine()